In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:
df=pd.read_json("tweets_data.json")
df

,token,pos,head,deprel,aspects
0,"[i, agree, about, arafat, ,, i, mean, ,, shit,...","[LS, VBP, IN, NN, ,, FW, NN, ,, NN, ,, PRP, RB...","[2, 0, 4, 2, 7, 7, 2, 7, 2, 2, 13, 13, 2, 13, ...","[nsubj, root, case, nmod, punct, nsubj, parata...","[{'term': ['jimmy', 'carter'], 'from': 15, 'to..."
1,"[musicmonday, britney, spears, -, lucky, do, y...","[NN, NN, NNS, :, JJ, VBP, PRP, VB, DT, NN, ,, ...","[3, 3, 0, 3, 3, 8, 8, 3, 10, 8, 8, 8, 8, 15, 3...","[amod, compound, root, punct, dep, aux, nsubj,...","[{'term': ['britney', 'spears'], 'from': 1, 't..."
2,"[wtf, ,, hilary, swank, is, coming, to, my, sc...","[NN, ,, JJ, NN, VBZ, VBG, TO, PRP$, NN, NN, ,,...","[6, 6, 4, 6, 6, 0, 9, 9, 6, 6, 6, 14, 14, 6, 1...","[advmod, punct, compound, nsubj, aux, root, ca...","[{'term': [',', 'hilary', 'swank'], 'from': 1,..."
3,"[my, 3-year-old, was, amazed, yesterday, to, f...","[PRP$, NN, VBD, VBN, NN, TO, VB, IN, '', JJ, '...","[2, 4, 4, 0, 4, 7, 4, 16, 10, 14, 14, 14, 14, ...","[nmod, nsubjpass, auxpass, root, nmod, mark, x...","[{'term': ['wii'], 'from': 21, 'to': 22, 'pola..."
4,"[God, damn, ,, That, Sony, remote, for, google...","[NNP, RB, ,, IN, NNP, JJ, IN, NN, VBZ, RB, JJ, .]","[0, 1, 1, 6, 6, 1, 8, 6, 10, 11, 1, 1]","[root, advmod, punct, mark, nsubj, amod, case,...","[{'term': ['google'], 'from': 7, 'to': 8, 'pol..."
...,...,...,...,...,...
6046,"[really, ,, that, seems, like, more, of, a, br...","[RB, ,, IN, VBZ, IN, JJR, IN, DT, NN, VBZ, NN,...","[4, 4, 4, 0, 6, 4, 11, 11, 11, 11, 6, 13, 11, ...","[advmod, punct, nsubj, root, case, nmod, case,...","[{'term': ['britney', 'spears'], 'from': 8, 't..."
6047,"[the, biggest, loser, wii, review]","[DT, JJS, NN, NN, NN]","[5, 5, 5, 5, 0]","[det, amod, amod, compound, root]","[{'term': ['wii'], 'from': 3, 'to': 4, 'polari..."
6048,"[ok, love, game, is, on, ,, ,, lady, gaga, ,, ...","[NN, NN, NN, VBZ, IN, ,, ,, NN, NNS, ,, PRP$, ...","[3, 3, 15, 15, 9, 9, 9, 9, 15, 9, 12, 9, 9, 15...","[amod, compound, nsubj, cop, case, punct, punc...","[{'term': [',', ',', 'lady', 'gaga'], 'from': ..."
6049,"[nic, cage, sues, business, manager, for, ', f...","[JJ, NN, VBZ, NN, NN, IN, '', JJ, VB, '', :, V...","[2, 3, 0, 5, 3, 9, 9, 9, 3, 9, 9, 9, 12, 18, 1...","[compound, nsubj, root, compound, dobj, case, ...","[{'term': ['nicolas', 'cage'], 'from': 16, 'to..."


In [3]:
df['text'] = df['token'].apply(lambda x: ' '.join(x))
df['label'] = df['aspects'].apply(lambda x: x[0]['polarity'] if x else 'neutral')
df

,token,pos,head,deprel,aspects,text,label
0,"[i, agree, about, arafat, ,, i, mean, ,, shit,...","[LS, VBP, IN, NN, ,, FW, NN, ,, NN, ,, PRP, RB...","[2, 0, 4, 2, 7, 7, 2, 7, 2, 2, 13, 13, 2, 13, ...","[nsubj, root, case, nmod, punct, nsubj, parata...","[{'term': ['jimmy', 'carter'], 'from': 15, 'to...","i agree about arafat , i mean , shit , they ev...",negative
1,"[musicmonday, britney, spears, -, lucky, do, y...","[NN, NN, NNS, :, JJ, VBP, PRP, VB, DT, NN, ,, ...","[3, 3, 0, 3, 3, 8, 8, 3, 10, 8, 8, 8, 8, 15, 3...","[amod, compound, root, punct, dep, aux, nsubj,...","[{'term': ['britney', 'spears'], 'from': 1, 't...",musicmonday britney spears - lucky do you reme...,positive
2,"[wtf, ,, hilary, swank, is, coming, to, my, sc...","[NN, ,, JJ, NN, VBZ, VBG, TO, PRP$, NN, NN, ,,...","[6, 6, 4, 6, 6, 0, 9, 9, 6, 6, 6, 14, 14, 6, 1...","[advmod, punct, compound, nsubj, aux, root, ca...","[{'term': [',', 'hilary', 'swank'], 'from': 1,...","wtf , hilary swank is coming to my school toda...",neutral
3,"[my, 3-year-old, was, amazed, yesterday, to, f...","[PRP$, NN, VBD, VBN, NN, TO, VB, IN, '', JJ, '...","[2, 4, 4, 0, 4, 7, 4, 16, 10, 14, 14, 14, 14, ...","[nmod, nsubjpass, auxpass, root, nmod, mark, x...","[{'term': ['wii'], 'from': 21, 'to': 22, 'pola...",my 3-year-old was amazed yesterday to find tha...,neutral
4,"[God, damn, ,, That, Sony, remote, for, google...","[NNP, RB, ,, IN, NNP, JJ, IN, NN, VBZ, RB, JJ, .]","[0, 1, 1, 6, 6, 1, 8, 6, 10, 11, 1, 1]","[root, advmod, punct, mark, nsubj, amod, case,...","[{'term': ['google'], 'from': 7, 'to': 8, 'pol...","God damn , That Sony remote for google is fuck...",negative
...,...,...,...,...,...,...,...
6046,"[really, ,, that, seems, like, more, of, a, br...","[RB, ,, IN, VBZ, IN, JJR, IN, DT, NN, VBZ, NN,...","[4, 4, 4, 0, 6, 4, 11, 11, 11, 11, 6, 13, 11, ...","[advmod, punct, nsubj, root, case, nmod, case,...","[{'term': ['britney', 'spears'], 'from': 8, 't...","really , that seems like more of a britney spe...",neutral
6047,"[the, biggest, loser, wii, review]","[DT, JJS, NN, NN, NN]","[5, 5, 5, 5, 0]","[det, amod, amod, compound, root]","[{'term': ['wii'], 'from': 3, 'to': 4, 'polari...",the biggest loser wii review,negative
6048,"[ok, love, game, is, on, ,, ,, lady, gaga, ,, ...","[NN, NN, NN, VBZ, IN, ,, ,, NN, NNS, ,, PRP$, ...","[3, 3, 15, 15, 9, 9, 9, 9, 15, 9, 12, 9, 9, 15...","[amod, compound, nsubj, cop, case, punct, punc...","[{'term': [',', ',', 'lady', 'gaga'], 'from': ...","ok love game is on , , lady gaga , my dear , y...",positive
6049,"[nic, cage, sues, business, manager, for, ', f...","[JJ, NN, VBZ, NN, NN, IN, '', JJ, VB, '', :, V...","[2, 3, 0, 5, 3, 9, 9, 9, 3, 9, 9, 9, 12, 18, 1...","[compound, nsubj, root, compound, dobj, case, ...","[{'term': ['nicolas', 'cage'], 'from': 16, 'to...",nic cage sues business manager for ' financial...,neutral


In [4]:
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

In [6]:
max_sequence_length = max(len(x) for x in sequences)
X = pad_sequences(sequences, maxlen=max_sequence_length)

In [7]:
X.shape

(6051, 35)

In [8]:
y=df['encoded_label']
y.shape

(6051,)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_sequence_length))
model.add(SimpleRNN(35, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [27]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 35, 128)           1539200   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 35)                5740      
                                                                 
 dropout_4 (Dropout)         (None, 35)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                2304      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
Total params: 1,547,439
Trainable params: 1,547,439
No

In [29]:
model_history = model.fit(X, y, epochs=10)

Epoch 1/10
190/190 [==============================] - 5s 22ms/step - loss: 1.0590 - accuracy: 0.4728
Epoch 2/10
190/190 [==============================] - 4s 22ms/step - loss: 0.9186 - accuracy: 0.5698
Epoch 3/10
190/190 [==============================] - 4s 22ms/step - loss: 0.5580 - accuracy: 0.7900
Epoch 4/10
190/190 [==============================] - 4s 22ms/step - loss: 0.2289 - accuracy: 0.9248
Epoch 5/10
190/190 [==============================] - 4s 22ms/step - loss: 0.1101 - accuracy: 0.9631
Epoch 6/10
190/190 [==============================] - 4s 22ms/step - loss: 0.0775 - accuracy: 0.9754
Epoch 7/10
190/190 [==============================] - 4s 22ms/step - loss: 0.0518 - accuracy: 0.9850
Epoch 8/10
190/190 [==============================] - 4s 22ms/step - loss: 0.0388 - accuracy: 0.9888
Epoch 9/10
190/190 [==============================] - 4s 22ms/step - loss: 0.0327 - accuracy: 0.9911
Epoch 10/10
190/190 [==============================] - 4s 22ms/step - loss: 0.0354 - accura

In [40]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

38/38 [==============================] - 0s 4ms/step - loss: 0.0038 - accuracy: 0.9992
Test accuracy: 0.9992


In [15]:
new_texts = ["I love this product", "very bad product", "It's okay, not bad"]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_padded)
predicted_classes = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

for text, sentiment in zip(new_texts, predicted_classes):
    print(f"Text: {text} --> Sentiment: {sentiment}")

1/1 [==============================] - 0s 237ms/step
Text: I love this product --> Sentiment: positive
Text: very bad product --> Sentiment: negative
Text: It's okay, not bad --> Sentiment: neutral


# Testing test dataset

In [16]:
testData=pd.read_json("test.json")
testData

,token,pos,head,deprel,aspects
0,"[shaquille, o'neal, to, miss, 3rd, straight, p...","[NN, NN, TO, VB, JJ, JJ, NN, NN, SYM, DT, ,, ,...","[2, 0, 4, 2, 8, 8, 8, 4, 10, 8, 10, 10, 10, 10...","[nsubj, root, mark, xcomp, compound, amod, com...","[{'term': ['shaquille', 'o'neal'], 'from': 0, ..."
1,"[Dear, justin, ,, Gray, Hoodies, turned, into,...","[NNP, NNP, ,, NNP, NNP, VBD, IN, NNP, NNPS, ,,...","[2, 6, 2, 5, 2, 0, 9, 9, 6, 6, 6, 6, 6, 6, 6, ...","[compound, nsubj, punct, compound, appos, root...","[{'term': ['justin'], 'from': 1, 'to': 2, 'pol..."
2,"[received, my, google, wave, account, today, ,...","[VBN, PRP$, NN, NN, NN, NN, ,, JJ, VBP, DT, VB...","[9, 5, 5, 5, 1, 1, 9, 9, 0, 11, 9, 9, 9, 16, 1...","[advcl, nmod, compound, compound, dobj, nmod, ...","[{'term': ['google', 'wave'], 'from': 2, 'to':..."
3,"[epascarello, I, know, ,, Man, I, get, pissed,...","[NN, PRP, VBP, ,, NNP, PRP, VBP, VBN, WRB, PRP...","[3, 3, 8, 3, 8, 8, 8, 0, 11, 11, 8, 13, 11, 15...","[advmod, nsubj, advcl, punct, nsubjpass, nsubj...","[{'term': ['google'], 'from': 16, 'to': 17, 'p..."
4,"[Is, it, just, me, ,, or, does, john, boehner,...","[VBZ, PRP, RB, PRP, ,, CC, VBZ, NN, JJ, NN, IN...","[15, 1, 4, 1, 1, 1, 10, 9, 10, 15, 13, 13, 10,...","[aux, nsubj, advmod, nsubj, punct, cc, aux, co...","[{'term': ['john', 'boehner'], 'from': 7, 'to'..."
...,...,...,...,...,...
672,"[not, a, goddamn, thing, ,, you, got, me, list...","[RB, DT, JJ, NN, ,, PRP, VBD, PRP, VBG, TO, NN...","[4, 4, 4, 7, 7, 7, 0, 9, 7, 14, 14, 11, 11, 9,...","[neg, det, amod, ccomp, punct, nsubj, root, ns...","[{'term': ['britney', 'spears'], 'from': 12, '..."
673,"[barack, obama, made, the, bullies, stop, pick...","[NN, NN, VBD, DT, VBZ, VB, VBG, IN, PRP, .]","[2, 3, 0, 5, 6, 3, 6, 9, 7, 3]","[compound, nsubj, root, det, nsubj, ccomp, xco...","[{'term': ['barack', 'obama'], 'from': 0, 'to'..."
674,"[apple, has, better, windows, 7, support, than...","[NN, VBZ, JJR, NNS, CD, NN, IN, NN, VBZ, ,, CC...","[2, 0, 4, 2, 6, 2, 9, 9, 6, 2, 2, 14, 12, 15, ...","[nsubj, root, amod, dobj, nummod, dobj, mark, ...","[{'term': ['windows', '7'], 'from': 3, 'to': 5..."
675,"[2, hours, later, ,, no, longer, seeing, confu...","[CD, NNS, RB, ,, DT, JJR, VBG, JJ, JJ, NN, NNS...","[2, 3, 8, 8, 6, 8, 8, 0, 11, 11, 8, 8]","[nummod, nmod, advmod, punct, neg, advmod, nsu...","[{'term': ['steve', 'jobs'], 'from': 9, 'to': ..."


In [17]:
testData['text'] = testData['token'].apply(lambda x: ' '.join(x))
testData['polarity'] = testData['aspects'].apply(lambda x: x[0]['polarity'] if x else 'neutral')
testData.drop(columns=['deprel','pos','head','token','aspects'],inplace=True)
testData

,text,polarity
0,shaquille o'neal to miss 3rd straight playoff ...,negative
1,"Dear justin , Gray Hoodies turned into Leather...",positive
2,"received my google wave account today , sorry ...",neutral
3,"epascarello I know , Man I get pissed when I t...",neutral
4,"Is it just me , or does john boehner sound lik...",negative
...,...,...
672,"not a goddamn thing , you got me listening to ...",neutral
673,barack obama made the bullies stop picking on ...,negative
674,apple has better windows 7 support than sony d...,positive
675,"2 hours later , no longer seeing confused russ...",neutral


In [18]:
test_sequences=tokenizer.texts_to_sequences(testData['text'])

In [30]:
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)
predictions = model.predict(test_padded)
testData['predicted_classes'] = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

22/22 [==============================] - 0s 4ms/step


In [50]:
y_test=pd.DataFrame(label_encoder.fit_transform(testData['polarity']))
y_test

,0
0,0
1,2
2,1
3,1
4,0
...,...
672,1
673,0
674,2
675,1


In [51]:
test_loss, test_acc = model.evaluate(test_padded, y_test)
print(f"Test accuracy: {test_acc:.4f}")

22/22 [==============================] - 0s 4ms/step - loss: 2.3862 - accuracy: 0.5406
Test accuracy: 0.5406


In [31]:
testData

,text,polarity,predicted_classes
0,shaquille o'neal to miss 3rd straight playoff ...,negative,neutral
1,"Dear justin , Gray Hoodies turned into Leather...",positive,positive
2,"received my google wave account today , sorry ...",neutral,positive
3,"epascarello I know , Man I get pissed when I t...",neutral,neutral
4,"Is it just me , or does john boehner sound lik...",negative,negative
...,...,...,...
672,"not a goddamn thing , you got me listening to ...",neutral,neutral
673,barack obama made the bullies stop picking on ...,negative,neutral
674,apple has better windows 7 support than sony d...,positive,neutral
675,"2 hours later , no longer seeing confused russ...",neutral,negative


In [32]:
testData['polarity'].value_counts()

polarity
neutral     336
positive    172
negative    169
Name: count, dtype: int64

In [33]:
testData['predicted_classes'].value_counts()

predicted_classes
neutral     308
positive    209
negative    160
Name: count, dtype: int64